# Linear Regression
see: http://spark.apache.org/mllib/
'
compare: https://github.com/krishnaik06/Pyspark-With-Python/blob/main/Tutorial%208-Linear%20Regression%20With%20Pyspark.ipynb

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Regression').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/29 12:20:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df=spark.read.csv('tips.csv',header=True,inferSchema=True) #inferSchema --> age type turns from string to integer

In [4]:
df.show()
df.printSchema()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

## StringIndexer
Categorical to Ordinal numbers

In [5]:
### Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [6]:
indexer=StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [7]:
indexer=StringIndexer(inputCols=["smoker","day","time"],outputCols=["smoker_indexed","day_indexed",
                                                                  "time_index"])
df_r=indexer.fit(df_r).transform(df_r)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_index|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|       0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|       0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|       0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|       0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|       0.0|
|     26.88|3.12|  M

## VectorAssembler
assamble encoded features

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed',
                          'time_index'],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [9]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_index|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|       0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|       0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|       0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 

In [10]:
finalized_data=output.select("Independent Features","total_bill")
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



## Regression

In [11]:
from pyspark.ml.regression import LinearRegression

In [12]:
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])


In [13]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

21/11/29 12:20:30 WARN Instrumentation: [51b50bed] regParam is zero, which might cause numerical instability and overfitting.
21/11/29 12:20:31 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
21/11/29 12:20:31 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
21/11/29 12:20:31 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
regressor.coefficients, regressor.intercept

(DenseVector([3.1167, 3.4153, -1.0985, 2.822, -0.1153, -2.0912]),
 2.048528068168775)

In [15]:
print("LogisticRegression parameters:\n" + regressor.explainParams() + "\n")

LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features, current: Independent Features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: total_bill)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >

In [16]:
### Predictions
pred_results=regressor.evaluate(test_data)
pred_results.predictions.show()

/usr/local/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07|12.774966705302035|
|(6,[0,1],[1.45,2.0])|      9.55|13.398304393137984|
|(6,[0,1],[1.97,2.0])|     12.02|15.018982381511453|
|(6,[0,1],[2.01,2.0])|     20.23| 15.14364991907864|
|(6,[0,1],[2.31,3.0])|     18.69|19.493945494911852|
|(6,[0,1],[2.34,4.0])|     17.81|23.002735192166533|
| (6,[0,1],[3.0,4.0])|     20.45|25.059749562025168|
|(6,[0,1],[3.18,2.0])|     19.82|18.790175392918947|
|(6,[0,1],[3.27,2.0])|     17.78|19.070677352445124|
|(6,[0,1],[3.35,3.0])|     20.65| 22.73530147165879|
| (6,[0,1],[4.3,2.0])|      21.7|22.280866444800264|
|[1.0,2.0,0.0,1.0,...|      12.6|14.817786995066516|
|[1.0,2.0,1.0,1.0,...|      5.75|13.373335353482787|
|[1.25,2.0,1.0,0.0...|      8.51| 9.354693031592538|
|[1.32,2.0,0.0,0.0...|      9.68|12.877805526940925|
|[1.36,3.0,1.0,0.0...|     18.64|13.1128178039

In [17]:
### Performance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

(0.4797767481799341, 3.5638208485479517, 24.281584682241192)

## Pipeline

In [18]:
from pyspark.ml import Pipeline

In [38]:
#reload data
df=spark.read.csv('tips.csv',header=True,inferSchema=True) #inferSchema --> age type turns from string to integer

In [39]:
##train test split
train_data,test_data=df.randomSplit([0.75,0.25])

In [40]:
indexer=StringIndexer(inputCols=["sex", "smoker","day","time"],outputCols=["sex_indexed", "smoker_indexed","day_indexed","time_index"])
assembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_index'],outputCol="Independent Features")
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill', probabilityCol='probability')

#onehotencoder_qualification_vector = OneHotEncoder(inputCol="qualificationIndex", outputCol="qualification_vec")
#onehotencoder_gender_vector = OneHotEncoder(inputCol="genderIndex", outputCol="gender_vec")
#onehotencoder_age_vector = OneHotEncoder(inputCol="ageIndex", outputCol="age_vec")#Create pipeline and pass all stages

pipeline = Pipeline(stages=[indexer,
                            assembler,
                            regressor,
                    ])

TypeError: __init__() got an unexpected keyword argument 'probabilityCol'

In [41]:
#run pipeline
model = pipeline.fit(train_data)
df_predicted = model.transform(test_data)
df_predicted.show()

21/11/29 12:30:36 WARN Instrumentation: [e68a22e7] regParam is zero, which might cause numerical instability and overfitting.


+----------+----+------+------+----+------+----+-----------+--------------+-----------+----------+--------------------+------------------+
|total_bill| tip|   sex|smoker| day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_index|Independent Features|        prediction|
+----------+----+------+------+----+------+----+-----------+--------------+-----------+----------+--------------------+------------------+
|      7.25| 1.0|Female|    No| Sat|Dinner|   1|        1.0|           0.0|        0.0|       0.0|[1.0,1.0,1.0,0.0,...| 7.684747809552794|
|      7.25|5.15|  Male|   Yes| Sun|Dinner|   2|        0.0|           1.0|        1.0|       0.0|[5.15,2.0,0.0,1.0...| 27.17790164142385|
|      7.51| 2.0|  Male|    No|Thur| Lunch|   2|        0.0|           0.0|        2.0|       1.0|[2.0,2.0,0.0,0.0,...| 12.79270857853788|
|      8.52|1.48|  Male|    No|Thur| Lunch|   2|        0.0|           0.0|        2.0|       1.0|[1.48,2.0,0.0,0.0...|11.128307027180613|
|      8.77| 2.0|  Male|   

In [42]:
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

(0.4797767481799341, 3.5638208485479517, 24.281584682241192)

In [43]:
# Make predictions on test documents and print columns of interest.
selected = df_predicted.select("total_bill", "tip", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row  # type: ignore
    print(
        "(%d, %s) --> prob=%s, prediction=%f" % (
            rid, text, str(prob), prediction   # type: ignore
        )
    )

AnalysisException: cannot resolve 'probability' given input columns: [Independent Features, day, day_indexed, prediction, sex, sex_indexed, size, smoker, smoker_indexed, time, time_index, tip, total_bill];
'Project [total_bill#1674, tip#1675, 'probability, prediction#1905]
+- Project [total_bill#1674, tip#1675, sex#1676, smoker#1677, day#1678, time#1679, size#1680, sex_indexed#1861, smoker_indexed#1862, day_indexed#1863, time_index#1864, Independent Features#1890, UDF(Independent Features#1890) AS prediction#1905]
   +- Project [total_bill#1674, tip#1675, sex#1676, smoker#1677, day#1678, time#1679, size#1680, sex_indexed#1861, smoker_indexed#1862, day_indexed#1863, time_index#1864, UDF(struct(tip, tip#1675, size_double_VectorAssembler_23f101a65a55, cast(size#1680 as double), sex_indexed, sex_indexed#1861, smoker_indexed, smoker_indexed#1862, day_indexed, day_indexed#1863, time_index, time_index#1864)) AS Independent Features#1890]
      +- Project [total_bill#1674, tip#1675, sex#1676, 

## One Hot Encoder

In [24]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["sex_indexed","time_index"],
                        outputCols=["sexVec1", "timeVec2"])
model = encoder.fit(df_r)
encoded = model.transform(df_r)
encoded.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+-------------+-------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_index|      sexVec1|     timeVec2|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+----------+-------------+-------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|       0.0|    (1,[],[])|(1,[0],[1.0])|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|(1,[0],[1.0])|(1,[0],[1.0])|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|       0.0|(1,[0],[1.0])|(1,[0],[1.0])|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|       0.0|(1,[0],[1.0])|(1,[0],[1.0])|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|       0.0|    (1,[],[])|(1,[0],

### With pipeline

In [25]:
#reload data
df=spark.read.csv('tips.csv',header=True,inferSchema=True) #inferSchema --> age type turns from string to integer
train_data,test_data=df.randomSplit([0.75,0.25])

In [26]:
indexer=StringIndexer(inputCols=["sex", "smoker","day","time"],outputCols=["sex_indexed", "smoker_indexed","day_indexed","time_index"])
encoder = OneHotEncoder(inputCols=["sex_indexed", "smoker_indexed","day_indexed","time_index"]
                        ,outputCols=["sex_vec", "smoker_vec","day_vec","time_vec"])
assembler=VectorAssembler(inputCols=['tip','size',"sex_vec", "smoker_vec","day_vec","time_vec"],outputCol="Independent Features")
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')

#onehotencoder_qualification_vector = OneHotEncoder(inputCol="qualificationIndex", outputCol="qualification_vec")
#onehotencoder_gender_vector = OneHotEncoder(inputCol="genderIndex", outputCol="gender_vec")
#onehotencoder_age_vector = OneHotEncoder(inputCol="ageIndex", outputCol="age_vec")#Create pipeline and pass all stages

pipeline = Pipeline(stages=[indexer,
                            encoder,
                            assembler,
                            regressor,
                    ])

In [27]:
#run pipeline
model = pipeline.fit(train_data)
df_predicted = model.transform(test_data)
df_predicted.show()

21/11/29 12:20:40 WARN Instrumentation: [231902eb] regParam is zero, which might cause numerical instability and overfitting.


+----------+----+------+------+----+------+----+-----------+--------------+-----------+----------+-------------+-------------+-------------+-------------+--------------------+------------------+
|total_bill| tip|   sex|smoker| day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_index|      sex_vec|   smoker_vec|      day_vec|     time_vec|Independent Features|        prediction|
+----------+----+------+------+----+------+----+-----------+--------------+-----------+----------+-------------+-------------+-------------+-------------+--------------------+------------------+
|      3.07| 1.0|Female|   Yes| Sat|Dinner|   1|        1.0|           1.0|        0.0|       0.0|    (1,[],[])|    (1,[],[])|(3,[0],[1.0])|(1,[0],[1.0])|(8,[0,1,4,7],[1.0...| 9.923824112756874|
|      7.74|1.44|  Male|   Yes| Sat|Dinner|   2|        0.0|           1.0|        0.0|       0.0|(1,[0],[1.0])|    (1,[],[])|(3,[0],[1.0])|(1,[0],[1.0])|[1.44,2.0,1.0,0.0...|15.748836464820725|
|      8.52|1.48|  Male| 